### Environment Settings

In [1]:
import joblib
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [2]:
def get_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate temporal feature metrics for each item/store in a sales DataFrame.
    
    For each sales entry of an item at a store, the function calculates:
    - Lag values for 1 day.
    - Cumulative sums for the last 3 and 7 days.
    - Rolling mean for 7 and 30 days.
    - Rolling standard deviation for 7 and 30 days.
    
    Parameters:
    ----------
    df : pd.DataFrame
        DataFrame containing sales data, with the following columns:
        - 'item_id': Item identifier.
        - 'store_id': Store identifier.
        - 'date': Sales date.
        - 'y': Sales value (target variable).
    
    Returns:
    --------
    pd.DataFrame
        Original DataFrame with new feature columns added.
    """
    # Sort the DataFrame by item_id, store_id, and date
    df = df.sort_values(['item_id', 'store_id', 'date'])

    # Calculate lag_1
    df['lag_1'] = df.groupby(['item_id', 'store_id'], observed=False)['y'].shift(1)
    
    # Calculate cumulative sums for lag_3 and lag_7 (not true lags)
    df['sum_3'] = sum(df.groupby(['item_id', 'store_id'], observed=False)['y'].shift(i) for i in range(1, 4))
    df['sum_7'] = sum(df.groupby(['item_id', 'store_id'], observed=False)['y'].shift(i) for i in range(1, 8))

    # Calculate rolling mean and standard deviation for 7 and 30 days
    for window in [7, 30]:
        df[f'rolling_mean_{window}'] = df.groupby(['item_id', 'store_id'], observed=False)['y'].rolling(window=window, min_periods=1).mean().shift(1).reset_index(level=[0, 1], drop=True)
        df[f'rolling_std_{window}'] = df.groupby(['item_id', 'store_id'], observed=False)['y'].rolling(window=window, min_periods=1).std().shift(1).reset_index(level=[0, 1], drop=True)
    
    return df

def add_date_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add day of the week and month features to the dataset, as categorical variables.

    Parameters:
    ----------
    df : pd.DataFrame
        DataFrame containing a 'date' column to extract the features from.

    Returns:
    --------
    pd.DataFrame
        DataFrame with new 'wday' and 'month' columns added as categorical variables.
    """
    # Day of the week (1 = Monday, 7 = Sunday)
    df['wday'] = df['date'].dt.dayofweek + 1
    df['wday'] = df['wday'].astype('category')

    # Month of the year
    df['month'] = df['date'].dt.month
    df['month'] = df['month'].astype('category')
    
    return df


def drop_unnecessary_columns(df: pd.DataFrame, columns_to_drop: list) -> pd.DataFrame:
    """
    Drop unnecessary columns from the dataset.

    Parameters:
    ----------
    df : pd.DataFrame
        DataFrame from which columns will be dropped.
    columns_to_drop : list
        List of column names to drop from the DataFrame.

    Returns:
    --------
    pd.DataFrame
        DataFrame with specified columns removed.
    """
    return df.drop(columns=columns_to_drop, axis=1)


### Data Loading n' Preparation

In [3]:
train = pd.read_parquet('../data/train.snap.parquet')

### Feature Engineering

In [4]:
# Apply functions to train and test datasets
train = add_date_features(train)

# Creating an unique snap column to reduce dimensionality
train['snap'] = 0
train.loc[
    (train['store_id'].str[:2] == 'CA') & (train['snap_CA'] == 1) |
    (train['store_id'].str[:2] == 'TX') & (train['snap_TX'] == 1) |
    (train['store_id'].str[:2] == 'WI') & (train['snap_WI'] == 1),
    'snap'
] = 1
train['snap'] = train['snap'].astype('category')

# Removing uneeded columns
columns_to_drop = ['id', 'dept_id', 'cat_id', 'state_id', 'wm_yr_wk', 'event_type_1', 
                   'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']
train = drop_unnecessary_columns(train, columns_to_drop)

train.head()

,item_id,store_id,d,y,date,event_name_1,sell_price,wday,month,snap
0,FOODS_1_001,CA_1,d_1542,1.0,2015-04-19,nan,2.240234,7,4,0
1,FOODS_1_001,CA_1,d_1543,0.0,2015-04-20,nan,2.240234,1,4,0
2,FOODS_1_001,CA_1,d_1544,0.0,2015-04-21,nan,2.240234,2,4,0
3,FOODS_1_001,CA_1,d_1545,0.0,2015-04-22,nan,2.240234,3,4,0
4,FOODS_1_001,CA_1,d_1546,1.0,2015-04-23,nan,2.240234,4,4,0


In [5]:
train = get_metrics(train)

cutoff_date = train['date'].min() + pd.Timedelta('30D')
train  = train[train['date'] > cutoff_date]

In [6]:
train.head()

,item_id,store_id,d,y,date,event_name_1,sell_price,wday,month,snap,lag_1,sum_3,sum_7,rolling_mean_7,rolling_std_7,rolling_mean_30,rolling_std_30
5718316,HOBBIES_1_001,CA_1,d_1573,0.0,2015-05-20,nan,8.257812,3,5,0,0.0,0.0,0.0,0.0,0.0,0.366667,0.668675
5718317,HOBBIES_1_001,CA_1,d_1574,0.0,2015-05-21,nan,8.257812,4,5,0,0.0,0.0,0.0,0.0,0.0,0.333333,0.660895
5718318,HOBBIES_1_001,CA_1,d_1575,0.0,2015-05-22,nan,8.257812,5,5,0,0.0,0.0,0.0,0.0,0.0,0.333333,0.660895
5718319,HOBBIES_1_001,CA_1,d_1576,0.0,2015-05-23,nan,8.257812,6,5,0,0.0,0.0,0.0,0.0,0.0,0.266667,0.583292
5718320,HOBBIES_1_001,CA_1,d_1577,0.0,2015-05-24,nan,8.257812,7,5,0,0.0,0.0,0.0,0.0,0.0,0.233333,0.568321


In [7]:
train.tail()

,item_id,store_id,d,y,date,event_name_1,sell_price,wday,month,snap,lag_1,sum_3,sum_7,rolling_mean_7,rolling_std_7,rolling_mean_30,rolling_std_30
5718280,FOODS_3_827,WI_3,d_1937,0.0,2016-05-18,nan,1.0,3,5,0,4.0,9.0,12.0,1.714286,2.058663,0.833333,1.288767
5718281,FOODS_3_827,WI_3,d_1938,2.0,2016-05-19,nan,1.0,4,5,0,0.0,9.0,12.0,1.714286,2.058663,0.833333,1.288767
5718282,FOODS_3_827,WI_3,d_1939,2.0,2016-05-20,nan,1.0,5,5,0,2.0,6.0,14.0,2.000000,1.914854,0.900000,1.295882
5718283,FOODS_3_827,WI_3,d_1940,5.0,2016-05-21,nan,1.0,6,5,0,2.0,4.0,15.0,2.142857,1.864454,0.966667,1.299425
5718284,FOODS_3_827,WI_3,d_1941,1.0,2016-05-22,nan,1.0,7,5,0,5.0,9.0,18.0,2.571429,2.149197,1.133333,1.479360


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11226877 entries, 5718316 to 5718284
Data columns (total 17 columns):
 #   Column           Dtype         
---  ------           -----         
 0   item_id          category      
 1   store_id         category      
 2   d                category      
 3   y                float32       
 4   date             datetime64[ns]
 5   event_name_1     category      
 6   sell_price       float32       
 7   wday             category      
 8   month            category      
 9   snap             category      
 10  lag_1            float32       
 11  sum_3            float32       
 12  sum_7            float32       
 13  rolling_mean_7   float64       
 14  rolling_std_7    float64       
 15  rolling_mean_30  float64       
 16  rolling_std_30   float64       
dtypes: category(7), datetime64[ns](1), float32(5), float64(4)
memory usage: 824.5 MB


### Modeling

#### Fitting

In [9]:
features = [
    'item_id', 'store_id', 'wday', 'month', 'event_name_1', 'snap', 'sell_price', 
    'lag_1', 'sum_3', 'sum_7', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_30', 'rolling_std_30'
]

train_val_split_date = '2016-04-25' # Validation starts here 

val = train.copy()
val = val[val['date'] >= train_val_split_date]
val.reset_index(drop=True, inplace=True)

train = train[train['date'] < train_val_split_date]
train.reset_index(drop=True, inplace=True)


print(val.shape, train.shape)

(853720, 17) (10373157, 17)


In [10]:
print(train['date'].max(), val['date'].min())

2016-04-24 00:00:00 2016-04-25 00:00:00


In [11]:
X_train, y_train = train[features], train['y']
X_val, y_val = val[features], val['y']

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(10373157, 14) (10373157,)
(853720, 14) (853720,)


In [12]:
model = xgb.XGBRegressor(
    enable_categorical = True,
    early_stopping_rounds = 10,
    random_state=42 
)

In [13]:
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True)

[0]	validation_0-rmse:2.94372
[1]	validation_0-rmse:2.51943
[2]	validation_0-rmse:2.27817
[3]	validation_0-rmse:2.14457
[4]	validation_0-rmse:2.07038
[5]	validation_0-rmse:2.02942
[6]	validation_0-rmse:2.00652
[7]	validation_0-rmse:1.99155
[8]	validation_0-rmse:1.98265
[9]	validation_0-rmse:1.97750
[10]	validation_0-rmse:1.97492
[11]	validation_0-rmse:1.97137
[12]	validation_0-rmse:1.96685
[13]	validation_0-rmse:1.96611
[14]	validation_0-rmse:1.96431
[15]	validation_0-rmse:1.96337
[16]	validation_0-rmse:1.96308
[17]	validation_0-rmse:1.96275
[18]	validation_0-rmse:1.96174
[19]	validation_0-rmse:1.96195
[20]	validation_0-rmse:1.96166
[21]	validation_0-rmse:1.96165
[22]	validation_0-rmse:1.96112
[23]	validation_0-rmse:1.96029
[24]	validation_0-rmse:1.95896
[25]	validation_0-rmse:1.95752
[26]	validation_0-rmse:1.95723
[27]	validation_0-rmse:1.95735
[28]	validation_0-rmse:1.95655
[29]	validation_0-rmse:1.95668
[30]	validation_0-rmse:1.95729
[31]	validation_0-rmse:1.95761
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=10,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [16]:
# Save pkl for future usage
joblib.dump(model, '../models/xgb_model.pkl')

['../models/xgb_model.pkl']

In [14]:
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)

print(f'MAE in train set: {mae:.2f}')

MAE in train set: 0.91


In [15]:
y_pred = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)

print(f'MAE in validation set: {mae:.2f}')

MAE in validation set: 0.99


----

#### Predicting

In [21]:
train, test = pd.read_parquet('../data/train.snap.parquet'), pd.read_parquet('../data/test.snap.parquet')

In [22]:
df = pd.concat([train[train['date'] >= '2016-03-15'], test])
df.reset_index(drop=True, inplace=True)

# Apply functions to train and test datasets
df = add_date_features(df)

# Creating an unique snap column to reduce dimensionality
df['snap'] = 0
df.loc[
    (df['store_id'].str[:2] == 'CA') & (df['snap_CA'] == 1) |
    (df['store_id'].str[:2] == 'TX') & (df['snap_TX'] == 1) |
    (df['store_id'].str[:2] == 'WI') & (df['snap_WI'] == 1),
    'snap'
] = 1
df['snap'] = df['snap'].astype('category')

# Removing uneeded columns
columns_to_drop = ['id', 'dept_id', 'cat_id', 'state_id', 'wm_yr_wk', 'event_type_1', 
                   'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']
df = drop_unnecessary_columns(df, columns_to_drop)

df.head()

,item_id,store_id,d,y,date,event_name_1,sell_price,wday,month,snap
0,FOODS_1_001,CA_1,d_1873,0.0,2016-03-15,nan,2.240234,2,3,0
1,FOODS_1_001,CA_1,d_1874,0.0,2016-03-16,nan,2.240234,3,3,0
2,FOODS_1_001,CA_1,d_1875,2.0,2016-03-17,StPatricksDay,2.240234,4,3,0
3,FOODS_1_001,CA_1,d_1876,1.0,2016-03-18,nan,2.240234,5,3,0
4,FOODS_1_001,CA_1,d_1877,2.0,2016-03-19,nan,2.240234,6,3,0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2957530 entries, 0 to 2957529
Data columns (total 10 columns):
 #   Column        Dtype         
---  ------        -----         
 0   item_id       category      
 1   store_id      category      
 2   d             category      
 3   y             float32       
 4   date          datetime64[ns]
 5   event_name_1  category      
 6   sell_price    float32       
 7   wday          category      
 8   month         category      
 9   snap          category      
dtypes: category(7), datetime64[ns](1), float32(2)
memory usage: 70.7 MB


In [24]:
features = [
    'item_id', 'store_id', 'wday', 'month', 'event_name_1', 'snap', 'sell_price', 
    'lag_1', 'sum_3', 'sum_7', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_30', 'rolling_std_30'
]


data = df.copy()
days = [f'd_{c}' for c in range(1942, 1970)]
# Creating a loop to predict t+2 based on predicted values for t+1 and so on
for d in days:
    datetime = data[data['d'] == d]['date'].iloc[0]
    start_window = datetime - pd.Timedelta('40D') 
    end_window  = datetime

    batch = data[(data['date'] > start_window) & (data['date'] <= end_window)]
    batch = get_metrics(batch)

    X_test = batch[batch['d'] == d][features]
    X_test["item_id"] = X_test["item_id"].cat.set_categories(X_train["item_id"].cat.categories)
    X_test["store_id"] = X_test["store_id"].cat.set_categories(X_train["store_id"].cat.categories)
    X_test["event_name_1"] = X_test["event_name_1"].cat.set_categories(X_train["event_name_1"].cat.categories)
    X_test["snap"] = X_test["snap"].cat.set_categories(X_train["snap"].cat.categories)
    X_test["wday"] = X_test["wday"].cat.set_categories(X_train["wday"].cat.categories)
    X_test["month"] = X_test["month"].cat.set_categories(X_train["month"].cat.categories)
    
    y_pred = model.predict(X_test)
    data.loc[X_test.index, 'y'] = y_pred
   

In [25]:
test = test[['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd', 'y']]
test

,item_id,dept_id,cat_id,store_id,state_id,d,y
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,2
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,2
...,...,...,...,...,...,...,...
853715,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0
853716,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0
853717,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0
853718,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,5


In [26]:
data = data[['item_id', 'store_id', 'd', 'y']]
data.rename(columns={"y": "y_pred"}, inplace=True)
data

,item_id,store_id,d,y_pred
0,FOODS_1_001,CA_1,d_1873,0.000000
1,FOODS_1_001,CA_1,d_1874,0.000000
2,FOODS_1_001,CA_1,d_1875,2.000000
3,FOODS_1_001,CA_1,d_1876,1.000000
4,FOODS_1_001,CA_1,d_1877,2.000000
...,...,...,...,...
2957525,FOODS_3_823,WI_3,d_1969,0.320247
2957526,FOODS_3_824,WI_3,d_1969,0.099268
2957527,FOODS_3_825,WI_3,d_1969,0.534173
2957528,FOODS_3_826,WI_3,d_1969,1.257810


In [27]:
df_merged = pd.merge(test, data[['item_id', 'store_id', 'd', 'y_pred']], on=['item_id', 'store_id', 'd'], how='left')
df_merged.reset_index(drop=True, inplace=True)
df_merged

,item_id,dept_id,cat_id,store_id,state_id,d,y,y_pred
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,2,1.010610
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0,0.234389
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0,0.645859
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0,1.838272
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,2,1.267043
...,...,...,...,...,...,...,...,...
853715,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0,0.320247
853716,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0,0.099268
853717,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0,0.534173
853718,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,5,1.257810


In [28]:
mae = mean_absolute_error(df_merged['y'], df_merged['y_pred'])
print(f'Mean Absolute Error (MAE): {mae}')

Mean Absolute Error (MAE): 1.0891172885894775


In [29]:
for d in days:
    scope = df_merged.copy()
    scope = scope[scope['d'] == d]
    mae = mean_absolute_error(scope['y'], scope['y_pred'])
    print(f'Mean Absolute Error (MAE) for {d}: {mae}')
    
    

Mean Absolute Error (MAE) for d_1942: 0.9335069060325623
Mean Absolute Error (MAE) for d_1943: 0.9030919671058655
Mean Absolute Error (MAE) for d_1944: 0.8924358487129211
Mean Absolute Error (MAE) for d_1945: 0.9069361090660095
Mean Absolute Error (MAE) for d_1946: 0.9716405272483826
Mean Absolute Error (MAE) for d_1947: 1.1184338331222534
Mean Absolute Error (MAE) for d_1948: 1.090580701828003
Mean Absolute Error (MAE) for d_1949: 1.0353870391845703
Mean Absolute Error (MAE) for d_1950: 0.931897759437561
Mean Absolute Error (MAE) for d_1951: 1.0162025690078735
Mean Absolute Error (MAE) for d_1952: 1.0499612092971802
Mean Absolute Error (MAE) for d_1953: 1.1846719980239868
Mean Absolute Error (MAE) for d_1954: 1.2627131938934326
Mean Absolute Error (MAE) for d_1955: 1.2934865951538086
Mean Absolute Error (MAE) for d_1956: 1.1449732780456543
Mean Absolute Error (MAE) for d_1957: 1.067493200302124
Mean Absolute Error (MAE) for d_1958: 1.045973539352417
Mean Absolute Error (MAE) for d_195

In [31]:
# Saving final predictions for test set

In [34]:
df_pivot = df_merged.pivot(index=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                    columns='d', 
                    values='y_pred').reset_index()
df_pivot.columns.name = None
df_pivot

,item_id,dept_id,cat_id,store_id,state_id,d_1942,d_1943,d_1944,d_1945,d_1946,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1.010610,0.994665,0.900883,0.921823,0.962425,...,1.014364,1.391640,1.395215,1.035418,0.830094,0.793835,0.953556,0.994157,1.296287,1.300778
1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_2,CA,0.783046,0.644460,0.581457,0.612728,0.621349,...,0.608956,0.871101,0.788443,0.544315,0.545798,0.486078,0.526569,0.539430,0.613900,0.588127
2,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_3,CA,0.952181,0.915054,0.784600,0.725045,0.765647,...,0.884188,1.106596,1.293601,0.993476,0.688637,0.710286,0.718453,0.838545,0.968679,0.942907
3,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_4,CA,0.875576,0.685296,0.696810,0.797241,0.837842,...,0.765280,0.880396,0.959887,0.802602,0.772105,0.692614,0.705365,0.745966,0.880396,0.854624
4,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_1,TX,0.600852,0.473924,0.410620,0.409910,0.377695,...,0.459637,0.514848,0.514848,0.456095,0.406284,0.425598,0.419035,0.396332,0.495535,0.309644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_827,FOODS_3,FOODS,TX_2,TX,0.476735,0.312655,0.312655,0.347443,0.297505,...,0.205997,0.326016,0.326016,0.202456,0.157419,0.176733,0.187726,0.258893,0.359599,0.324199
30486,FOODS_3_827,FOODS_3,FOODS,TX_3,TX,2.082200,2.033165,2.008923,1.507046,1.894614,...,2.202595,2.209961,2.564006,2.235283,1.947256,2.030094,1.977562,2.217911,2.604542,2.482075
30487,FOODS_3_827,FOODS_3,FOODS,WI_1,WI,2.929816,2.586137,2.779960,2.936124,3.186957,...,3.338991,4.396729,4.296562,3.230777,3.268942,3.223502,2.961119,3.307161,4.224660,3.717051
30488,FOODS_3_827,FOODS_3,FOODS,WI_2,WI,2.405533,2.237345,2.352230,2.395033,2.265495,...,2.640103,3.171265,3.170766,2.261916,2.553440,2.565937,2.415069,2.640103,3.388150,3.163589


In [39]:
df_pivot.to_csv('../submission/predictions.csv', index=False)